# Google Earth Engine (GEE) DisALEXI 
## Mexico 2015 - Landsat Collection 1 TOA

Example of calling the GEE DisALEXI function on a mosaiced (merging rows) Landsat 8 Collection 1 top-of-atmosphere reflectance image.  
Study area WRS2 tiles: p039r037, p038r037, p039r038, p038r038

https://console.cloud.google.com/storage/browser/gcp-public-data-landsat/LC08/01/038/038/LC08_L1TP_038038_20150717_20170226_01_T1/

In [1]:
import ee
from IPython.display import Image

# For some reason the initialize has to be before the model imports below
ee.Initialize()

import openet.disalexi.disalexi as disalexi
import openet.disalexi.landsat as landsat
import openet.disalexi.utils as utils

ndvi_palette = ['#EFE7E1', '#003300']
ssebop_et_palette = [
  'DEC29B', 'E6CDA1', 'EDD9A6', 'F5E4A9', 'FFF4AD', 'C3E683', '6BCC5C', 
  '3BB369', '20998F', '1C8691', '16678A', '114982', '0B2C7A']

### Study area

In [2]:
output_extent = [624000, 3535000, 714000, 3625000]
output_crs = 'EPSG:32611'
cell_size = 30
output_transform = [cell_size, 0, output_extent[0], 0, -cell_size, output_extent[3]]
output_transform_str = '[' + ','.join(map(str, output_transform)) + ']'
output_shape_str = '{0}x{1}'.format(
    int(abs(output_extent[2] - output_extent[0]) / cell_size),
    int(abs(output_extent[3] - output_extent[1]) / cell_size))
output_region = ee.Geometry.Rectangle(output_extent, output_crs, False) \
    .bounds(1, 'EPSG:4326').coordinates().getInfo()
print(output_transform_str)
print(output_shape_str)

[30,0,624000,0,-30,3625000]
3000x3000


In [3]:
thumbnail_region = [[-115.09, 32.32], [-115.09, 32.40], [-114.95, 32.40], [-114.95, 32.32]]
thumbnail_geom = ee.Geometry.Polygon(thumbnail_region)

### Prep DisALEXI Input

In [4]:
scene_id = 'LC08_038XXX_20150717'
landsat_a_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_038037_20150717')
landsat_b_img = ee.Image('LANDSAT/LC08/C01/T1_RT_TOA/LC08_038038_20150717')

# Get the projection information from the first image
image_crs = landsat_a_img.select('B2').projection().crs().getInfo()
image_transform = landsat_a_img.select('B2').projection().getInfo()['transform']
image_geom = landsat_a_img.select('B2').geometry()
print(image_crs)
print(image_transform)

# Mosaic images into a single Landsat image
landsat_img = ee.Image(ee.ImageCollection([landsat_a_img, landsat_b_img]).mosaic()) \
    .reproject(crs=image_crs, crsTransform=image_transform)

# Copy the image properties from the first image
landsat_img = ee.Image(landsat_img.copyProperties(landsat_a_img, [
    'system:time_start', 'system:index', 'SPACECRAFT_ID', 
    'K1_CONSTANT_BAND_10', 'K2_CONSTANT_BAND_10']))

input_img = ee.Image(landsat.Landsat(landsat_img).prep())

EPSG:32611
[30.0, 0.0, 659985.0, 0.0, -30.0, 3789615.0]


### NDVI 

In [5]:
Image(url=input_img.select(['ndvi']).paint(thumbnail_geom, 'red', 1).getThumbURL({
    'region': output_region, 'min': -0.1, 'max': 0.9, 
    'palette': ','.join(ndvi_palette)}))

### Apply DisALEXI

In [6]:
# Use the CONUS ALEXI ET but the global landcover and elevation products
disalexi = disalexi.Image(
    input_img, 
    iterations=10,
    elevation=ee.Image('USGS/SRTMGL1_003').rename(['elevation']),
    landcover=ee.Image(
        ee.ImageCollection('users/cgmorton/GlobeLand30').mosaic()) \
            .divide(10).floor().multiply(10).rename(['landcover']),
    lc_type='GLOBELAND30')

ta_img = disalexi.compute_ta() \
    .reproject(crs=output_crs, crsTransform=output_transform)
ta_coarse_img = disalexi.aggregate(ta_img)
# Eventually make smoothing to a DisALEXI method
ta_smooth_img = ta_coarse_img \
    .resample('bilinear') \
    .reproject(crs=output_crs, crsTransform=output_transform)
et_img = disalexi.compute_et(ta_smooth_img)

# et_img = disalexi.compute_et(ta_smooth_img) \
#     .reproject(crs=output_crs, crsTransform=output_transform)

In [7]:
# print(t_air_img.getInfo())
# print(t_air_coarse_img.getInfo())
# print(et_img.getInfo())

### Export to Drive

In [8]:
et_task = ee.batch.Export.image.toDrive(
    et_img.toFloat(),
    description=scene_id + '_et',
    folder='DisALEXI',
    fileNamePrefix=scene_id + '_et',
    dimensions=output_shape_str,
    crs=output_crs,
    crsTransform=output_transform_str,
    maxPixels=1E10)

In [9]:
et_task.start()
print('Status: {}'.format(et_task.status()))

Status: {'id': 'L7CM4TSOWAGOPNWSBIQEMETQ', 'state': 'READY', 'creation_timestamp_ms': 1538069922488, 'update_timestamp_ms': 1538069922488, 'description': 'LC08_038XXX_20150717_et', 'task_type': 'EXPORT_IMAGE'}


In [10]:
# ta_task = ee.batch.Export.image.toDrive(
#     ta_img.toFloat(),
#     description=scene_id + '_ta',
#     folder='DisALEXI',
#     fileNamePrefix=scene_id + '_ta',
#     dimensions=output_shape_str,
#     crs=output_crs,
#     crsTransform=output_transform_str,
#     maxPixels=1E10)

In [11]:
# ta_coarse_task = ee.batch.Export.image.toDrive(
#     ta_coarse_img.toFloat(),
#     description=scene_id + '_ta_coarse',
#     folder='DisALEXI',
#     fileNamePrefix=scene_id + '_ta_coarse',
#     dimensions=output_shape_str,
#     crs=output_crs,
#     crsTransform=output_transform_str,
#     maxPixels=1E10)

In [12]:
# ta_smooth_task = ee.batch.Export.image.toDrive(
#     ta_smooth_img.toFloat(),
#     description=scene_id + '_ta_smooth',
#     folder='DisALEXI',
#     fileNamePrefix=scene_id + '_ta_smooth',
#     dimensions=output_shape_str,
#     crs=output_crs,
#     crsTransform=output_transform_str,
#     maxPixels=1E10)

In [13]:
# ta_task.start()
# print('Status: {}'.format(ta_task.status()))
# ta_coarse_task.start()
# print('Status: {}'.format(ta_coarse_task.status()))
# ta_smooth_task.start()
# print('Status: {}'.format(ta_smooth_task.status()))

### Thumbnails

In [14]:
Image(url=input_img.select(['ndvi']).getThumbURL({
    'region': thumbnail_region, 'min': -0.1, 'max': 0.9, 
    'palette': ','.join(ndvi_palette)}))

In [15]:
alexi_url = ee.Image(disalexi.alexi_et).reproject(crs=output_crs, crsTransform=output_transform).getThumbURL({
    'region': thumbnail_region, 'min': 0, 'max': 10, 
    'palette': ','.join(['FF0000', 'FFFF00', '00FFFF', '0000FF'])})
# print(alexi_url)
Image(url=alexi_url)

In [16]:
ta_url = ta_img.getThumbURL({
    'region': thumbnail_region, 'min': 273, 'max': 325, 
    'palette': ','.join(['FF0000', 'FFFF00', '00FFFF', '0000FF'])})
# print(ta_url)
Image(url=ta_url)

In [17]:
# ta_coarse_url = ta_coarse_img.getThumbURL({
#     'region': thumbnail_region, 'min': 273, 'max': 325, 
#     'palette': ','.join(['FF0000', 'FFFF00', '00FFFF', '0000FF'])})
# # print(ta_coarse_url)
# Image(url=ta_coarse_url)

In [18]:
# ta_smooth_url = ta_smooth_img.getThumbURL({
#     'region': thumbnail_region, 'min': 273, 'max': 325, 
#     'palette': ','.join(['FF0000', 'FFFF00', '00FFFF', '0000FF'])})
# # print(ta_smooth_url)
# Image(url=ta_smooth_url)

In [19]:
# et_url = et_img.getThumbURL({
#     'region': thumbnail_region, 'min': 0, 'max': 10, 
#     'palette': ','.join(ssebop_et_palette)})
# # print(et_url)
# Image(url=et_url)